In [1]:
#%load_ext snakeviz
%matplotlib inline

In [2]:
#librerias utilitarias
from random import randint
import scipy.io as sio
import numpy as np
import matplotlib.pyplot as plt
import random

In [3]:
#librerias Keras
from keras.models import Sequential
from keras.layers.core import Dense, Dropout, Activation, Flatten
from keras.layers.convolutional import Convolution2D, MaxPooling2D, AveragePooling2D
from keras.optimizers import SGD, Adadelta, Adagrad
from keras.utils import np_utils
from keras import backend as K

Using TensorFlow backend.


In [4]:
from utils.multi_gpu import make_parallel

In [18]:
import pandas as pd

In [5]:
from utils.plot import plot_image

In [5]:
!nvidia-smi

Fri Jun 16 01:56:50 2017       
+-----------------------------------------------------------------------------+
| NVIDIA-SMI 375.39                 Driver Version: 375.39                    |
|-------------------------------+----------------------+----------------------+
| GPU  Name        Persistence-M| Bus-Id        Disp.A | Volatile Uncorr. ECC |
| Fan  Temp  Perf  Pwr:Usage/Cap|         Memory-Usage | GPU-Util  Compute M. |
|===============================+======================+======================|
|   0  Tesla K80           Off  | 0000:00:17.0     Off |                    0 |
| N/A   62C    P0    62W / 149W |      0MiB / 11439MiB |      0%      Default |
+-------------------------------+----------------------+----------------------+
|   1  Tesla K80           Off  | 0000:00:18.0     Off |                    0 |
| N/A   50C    P0    74W / 149W |      0MiB / 11439MiB |      0%      Default |
+-------------------------------+----------------------+----------------------+
|   2  T

### Cargar Datos
___

In [4]:
train_data = sio.loadmat('data/train_32x32.mat')
test_data = sio.loadmat('data/test_32x32.mat')
X_train = train_data['X'].T
y_train = train_data['y'] - 1
X_test = test_data['X'].T
y_test = test_data['y'] - 1
X_train = X_train.astype('float32')
X_test = X_test.astype('float32')
n_classes = len(np.unique(y_train))
#print (np.unique(y_train))
X_train /= 255
X_test /= 255
Y_train = np_utils.to_categorical(y_train, n_classes)
Y_test = np_utils.to_categorical(y_test, n_classes)
print (X_train.shape)
print (X_test.shape)
print (Y_train.shape)
print (Y_test.shape)

(73257, 3, 32, 32)
(26032, 3, 32, 32)
(73257, 10)
(26032, 10)


### Preparar paralelismo
___

Para parelizar la ejecución del modelo a nivel de datos se debe tener complir la siguiente relación:

$$ \frac{\frac{datos-de-entrenamiento}{batches}}{Nº GPU`s} = 0$$

Por quitamos 1 ejemplo de entreamiento y otro de test. 

In [5]:
Y_test_parallel =np.delete(Y_test, 1, 0)
X_test_parallel =np.delete(X_test, 1, 0)
Y_train_parallel =np.delete(Y_train, 1, 0)
X_train_parallel =np.delete(X_train, 1, 0)
print (X_train_parallel.shape)
print (X_test_parallel.shape)
print (Y_train_parallel.shape)
print (Y_test_parallel.shape)

(73256, 3, 32, 32)
(26031, 3, 32, 32)
(73256, 10)
(26031, 10)


Objetivo
-----

**(H)**Elija una de las redes entrenadas en esta seccion y determine los pares de dıgitos (por ejemplo “1” con “7”) que la red tiende a confundir. Conjeture el motivo de tal confusion.

In [6]:
from keras.models import load_model
model_completo = load_model('modeloCompleto.h5')

In [13]:
from keras.models import load_model
modelo = load_model('modeloBase.h5')

In [23]:
clases=['cero','uno','dos','tres','cuatro','cinco','seis','siete','ocho','nueve']

**Paso 1**: consiste en encontrar los índices donde están los ejemplos de cada categoría, para se implemento la función:

In [8]:
def buscarLabelCategoria_(clase,Yt_):
    indices=[]
    for i in range(len(Yt_)):
        if Yt_[i] == clase:
            indices.append(i)
    return np.array(indices)

In [9]:
indices_cero = buscarLabelCategoria_(0, y_test)
indices_uno = buscarLabelCategoria_(1, y_test)
indices_dos = buscarLabelCategoria_(2, y_test)
indices_tres = buscarLabelCategoria_(3, y_test)
indices_cuatro = buscarLabelCategoria_(4, y_test)
indices_cinco = buscarLabelCategoria_(5, y_test)
indices_seis = buscarLabelCategoria_(6, y_test)
indices_siete = buscarLabelCategoria_(7, y_test)
indices_ocho = buscarLabelCategoria_(8, y_test)
indices_nueve = buscarLabelCategoria_(9, y_test)

**Paso 2**: Ya con el listado de índices de cada categoría, se selecciona un subset de datos para cada categoría, para esto se implemento la función:

In [10]:
def seleccionarSubconjuntoClase_(listaIndices_,X_test_):
    dataSetClase=[]
    for i in listaIndices_:
        dataSetClase.append(X_test_[i])
    return np.array(dataSetClase)

In [15]:
dataset_cero = seleccionarSubconjuntoClase_(indices_cero,X_test)
dataset_uno = seleccionarSubconjuntoClase_(indices_uno,X_test)
dataset_dos = seleccionarSubconjuntoClase_(indices_dos,X_test)
dataset_tres = seleccionarSubconjuntoClase_(indices_tres,X_test)
dataset_cuatro = seleccionarSubconjuntoClase_(indices_cuatro,X_test)
dataset_cinco = seleccionarSubconjuntoClase_(indices_cinco,X_test)
dataset_seis = seleccionarSubconjuntoClase_(indices_seis,X_test)
dataset_siete = seleccionarSubconjuntoClase_(indices_siete,X_test)
dataset_ocho = seleccionarSubconjuntoClase_(indices_ocho,X_test)
dataset_nueve = seleccionarSubconjuntoClase_(indices_nueve,X_test)

**Paso 3**: Ahora leemos el modelo guardo, el que mostro el mejor acurracy, y aplicado el predicción a cada dataset de cada catagoria:

In [16]:
%%time
prediccion_cero= modelo.predict(dataset_cero, verbose=1)
prediccion_uno= modelo.predict(dataset_uno, verbose=0)
prediccion_dos= modelo.predict(dataset_dos, verbose=0)
prediccion_tres= modelo.predict(dataset_tres, verbose=0)
prediccion_cuatro= modelo.predict(dataset_cuatro, verbose=0)
prediccion_cinco= modelo.predict(dataset_cinco, verbose=0)
prediccion_seis= modelo.predict(dataset_seis, verbose=0)
prediccion_siete= modelo.predict(dataset_siete, verbose=0)
prediccion_ocho= modelo.predict(dataset_ocho, verbose=0)
prediccion_nueve= modelo.predict(dataset_nueve, verbose=1)

1632/1744 [===========================>..] - ETA: 0s

**Paso 4**: Desde en este paso en adelante, usaremos la API de Python Pandas que nos permitirá ordenar el resultado de obtenido desde la predicción de Keras. Para esto a partir de la salida de Keras(n-array) pasamos a un pandas dataframe. Y en la misma línea calculamos las mayores frecuencias por cada clase:


In [19]:
df_cero = pd.DataFrame(prediccion_cero).max()
df_uno = pd.DataFrame(prediccion_uno).max()
df_dos = pd.DataFrame(prediccion_dos).max()
df_tres = pd.DataFrame(prediccion_tres).max()
df_cuatro = pd.DataFrame(prediccion_cuatro).max()
df_cinco = pd.DataFrame(prediccion_cinco).max()
df_seis = pd.DataFrame(prediccion_seis).max()
df_siete = pd.DataFrame(prediccion_siete).max()
df_ocho = pd.DataFrame(prediccion_ocho).max()
df_nueve = pd.DataFrame(prediccion_nueve).max()

**Paso 5**: Finalmente aplicamos un sort descendente a la serie de pandas que fue la salida del punto anterior, para en el paso siguiente presentar los valores que tienden a confundirse por categoria:

In [20]:
df_cero.sort_values(inplace=True,ascending=False)
df_uno.sort_values(inplace=True,ascending=False)
df_dos.sort_values(inplace=True,ascending=False)
df_tres.sort_values(inplace=True,ascending=False)
df_cuatro.sort_values(inplace=True,ascending=False)
df_cinco.sort_values(inplace=True,ascending=False)
df_seis.sort_values(inplace=True,ascending=False)
df_siete.sort_values(inplace=True,ascending=False)
df_ocho.sort_values(inplace=True,ascending=False)
df_nueve.sort_values(inplace=True,ascending=False)

In [21]:
clases_cero = df_cero.index
clases_uno = df_uno.index
clases_dos = df_dos.index
clases_tres = df_tres.index
clases_cuatro = df_cuatro.index
clases_cinco = df_cinco.index
clases_seis = df_seis.index
clases_siete = df_siete.index
clases_ocho = df_ocho.index
clases_nueve = df_nueve.index

In [24]:
print("Para Cero, la predicciones suele confundir: ",clases[clases_cero[0]]," y " ,clases[clases_cero[1]])
print("Para Uno, la predicciones suele confundir: ",clases[clases_uno[0]]," y " ,clases[clases_uno[1]])
print("Para Dos, la predicciones suele confundir: ",clases[clases_dos[0]]," y " ,clases[clases_dos[1]])
print("Para Tres, la predicciones suele confundir: ",clases[clases_tres[0]]," y " ,clases[clases_tres[1]])
print("Para Cuatro, la predicciones suele confundir: ",clases[clases_cuatro[0]]," y " ,clases[clases_cuatro[1]])
print("Para Cinco, la predicciones suele confundir: ",clases[clases_cinco[0]]," y " ,clases[clases_cinco[1]])
print("Para Seis, la predicciones suele confundir: ",clases[clases_seis[0]]," y " ,clases[clases_seis[1]])
print("Para Siete, la predicciones suele confundir: ",clases[clases_siete[0]]," y " ,clases[clases_siete[1]])
print("Para Ocho, la predicciones suele confundir: ",clases[clases_ocho[0]]," y " ,clases[clases_ocho[1]])
print("Para Nueve, la predicciones suele confundir: ",clases[clases_nueve[0]]," y " ,clases[clases_nueve[1]])

Para Cero, la predicciones suele confundir:  cero  y  cuatro
Para Uno, la predicciones suele confundir:  uno  y  cero
Para Dos, la predicciones suele confundir:  dos  y  nueve
Para Tres, la predicciones suele confundir:  tres  y  cero
Para Cuatro, la predicciones suele confundir:  cuatro  y  nueve
Para Cinco, la predicciones suele confundir:  cinco  y  nueve
Para Seis, la predicciones suele confundir:  seis  y  cero
Para Siete, la predicciones suele confundir:  cero  y  siete
Para Ocho, la predicciones suele confundir:  ocho  y  uno
Para Nueve, la predicciones suele confundir:  nueve  y  ocho


Como podemos ver en la mayoría de las categorías, salvo para el numero siete los resultados de la red son coherentes con las categorías. Esto bajo el escenario que el modelo había alcanzado un accuracy cercano al 94% de accuracy. 

**Nota:**Se debe recordar lo indicado al principio de las actividades, que las clases en dataset de entrenamiento están sobre desbalanceadas para el 0 y 1, esto no es menor porque como podemos ver en muchos de los casos la red tiene como segunda opción a estos dígitos.